In [ ]:
## Master file lets test here everything
import os
from IPython import display
import imagingplus as ip
from imagingplus.processing.paq import PaqData
from imagingplus.processing.imagingMetadata import PrairieViewMetadata

In [ ]:
ExperimentMetainfo = {
    'dataPath': '/home/pshah/mnt/qnap/Data/2020-12-19',  #: main dir where the imaging data is contained
    'saveDir': '/mnt/qnap_share/Data/imagingplus-example/',  #: main dir where the experiment object and the trial objects will be saved to
    "exID": 'RL109',  #: given identification name for experiment
    'comment': 'date: 2020-12-19',  #: notes related to experiment
}

# create Experiment

expobj = ip.Experiment(**ExperimentMetainfo)


# saving the Experiment object can be done anytime to disk from `expobj.save()`:
expobj.save()
# to save the Experiment with a custom path, provide a custom .pkl path and use the .save_pkl() method:
#expobj.save_pkl(pkl_path='/mnt/qnap_share/Data/imagingplus-example/RL109_duplicate_analysis.pkl')

In [ ]:
# we can now load the new expobj from disk storage using pickle in ip.import_obj():
expobj = ip.import_obj('/mnt/qnap_share/Data/imagingplus-example/RL109_analysis.pkl')

# create trial obj for each trial of experiment

trials_list_spont = ['t-005', 't-006']
for idx, trial in enumerate(trials_list_spont):
    date = '2020-12-19'
    prep = expobj.expID

    # define paths to retrieve data from disc
    paqs_loc = f'/home/pshah/mnt/qnap/Data/2020-12-19/{date}_{prep}_{trial[-3:]}.paq'  # path to the .paq files for the selected trials
    dataPath = f'/home/pshah/mnt/qnap/Data/2020-12-19/{date}_{trial}/{date}_{trial}_Cycle00001_Ch3.tif'


    # initialize microscope meta-data for current imaging trial using pre-built module for Bruker-PrairieView xml parser
    imparams = PrairieViewMetadata(pv_xml_dir=os.path.dirname(dataPath), microscope='Bruker 2pPlus')

    # initialize temporal data using pre-built module for .paq parsing
    tmdata = PaqData.paqProcessingTwoPhotonImaging(paq_path=paqs_loc, frame_channel='frame_clock', plot=False)

    # feed in information into `TwoPhotonImaging` to create a TwoPhotonImaging object
    trialobj = ip.TwoPhotonImaging(date=date, trialID= trial, expID= prep, imparams =  imparams, tmdata= tmdata,
                                saveDir=f'/mnt/qnap_share/Data/imagingplus-example/imagingplus-test-analysis/',
                                dataPath= dataPath, expGroup= "awake spont. 2p imaging + LFP")


    # add each Trial to the overall Experiment using the trialobj
    expobj.add_imaging_trial(trialID=trial, trialobj=trialobj)
    
# save the experiment object after adding new trials
expobj.save()
print(expobj)

In [ ]:
# we can access each trial's information from within the `expobj`
expobj.TrialsInformation
# we can also load each trial directly from the experiment:
trialobj = expobj.load_trial(trialID = 't-005')